In [27]:
import json, os, requests
import load_model

In [21]:
os.getcwd()

'D:\\Data\\PyCharmProjects\\mlzoomcamp-certification\\5_deployment\\feedback'

In [11]:
!pipenv --version

pipenv, version 2022.7.4



C:\Users\oluse\anaconda3\envs\ml-zoomcamp\Lib\site-packages\pipenv\vendor\attr\_make.py:876: RuntimeWarning: Running interpreter doesn't sufficiently support code object introspection.  Some features like bare super() or accessing __class__ will not work with slotted classes.
  set_closure_cell(cell, cls)


In [17]:
# Path to your Pipfile.lock
pipfile_lock_path = 'D:\Data\PyCharmProjects\mlzoomcamp-certification\Pipfile.lock'

# Read and parse the Pipfile.lock file
with open(pipfile_lock_path, 'r') as file:
    pipfile_data = json.load(file)

# Search for the first hash for scikit-learn in both default and develop sections
scikit_learn_hash = None
for section in ['default', 'develop']:
    if 'scikit-learn' in pipfile_data.get(section, {}):
        scikit_learn_data = pipfile_data[section]['scikit-learn']
        scikit_learn_hash = scikit_learn_data['hashes'][0] if 'hashes' in scikit_learn_data else None
        break

# Output the result
if scikit_learn_hash:
    print("First hash for scikit-learn:", scikit_learn_hash)
# else:
#     print("scikit-learn not found in Pipfile.lock or no hashes available.")

First hash for scikit-learn: sha256:03b6158efa3faaf1feea3faa884c840ebd61b6484167c711548fce208ea09445


Download model and vectorizer

In [25]:
# URLs 
urls = [
    "https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework/model1.bin",
    "https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework/dv.bin"
]

In [23]:
# Download each file
for url in urls:
    filename = url.split("/")[-1]  # Get the filename from the URL
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded {filename}")
    else:
        print(f"Failed to download {filename}")

Downloaded model1.bin
Downloaded dv.bin


In [29]:
model, dv = load_model.load_models()

In [31]:
model

LogisticRegression(max_iter=250)

In [32]:
dv

DictVectorizer(sparse=False)

Score this client:

In [30]:
client = {"job": "management", "duration": 400, "poutcome": "success"}

In [34]:
X = dv.transform([client])
X

array([[400.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   1.,   0.]])

In [35]:
y_pred = model.predict_proba(X)[0, 1]

In [36]:
print('input:', client)
print('output:', y_pred)

input: {'job': 'management', 'duration': 400, 'poutcome': 'success'}
output: 0.7590966516879658


Now let's serve this model as a web service

In [39]:
url = 'http://localhost:9685/clientpredictor'

In [40]:
client = {"job": "student", "duration": 280, "poutcome": "failure"}
requests.post(url, json=client).json()

{'churn': False, 'churn_probability': 0.33480703475511053}

Docker

In [47]:
# docker pull svizor/zoomcamp-model:3.11.5-slim 

In [49]:
!docker images svizor/zoomcamp-model 

REPOSITORY              TAG           IMAGE ID       CREATED       SIZE
svizor/zoomcamp-model   3.11.5-slim   975e7bdca086   11 days ago   130MB


In [51]:
client = {"job": "management", "duration": 400, "poutcome": "success"}
requests.post(url, json=client).json()

{'churn': True, 'churn_probability': 0.756743795240796}